In [1]:
import orchest
import numpy as np
from pyclarify import *
from utils import Anomaly

In [2]:
data = orchest.get_inputs()

In [3]:
clarify_data = data['clarify_data']
energy_consumption_dfs = clarify_data['energy_consumption']

anomalies = data['anomalies_from_et']['anomalies_from_ett']

In [4]:
clarify_client = ClarifyClient("clarify-credentials.json")

In [5]:
building_name = list(energy_consumption_dfs['hourly'])[1]

energy_consumption_dfs['hourly'][building_name]

start = energy_consumption_dfs['hourly'][building_name].index[0]
end = energy_consumption_dfs['hourly'][building_name].index[-1]

In [6]:
anomalies[building_name].sort(key=lambda x: x.start)

# for a in anomalies[building_name]:
#     print(f'building: {a.building}    start: {a.start}    end: {a.end}')
    
print(len(anomalies[building_name]))

c = None
i = 0
while i < len(anomalies[building_name]):
    if anomalies[building_name][i].start == c:
         anomalies[building_name].pop(i)
    else:
        c = anomalies[building_name][i].start
        i += 1

print(len(anomalies[building_name]))

64
64


In [7]:
start

Timestamp('2019-01-01 00:00:00+0000', tz='UTC')

In [8]:
end

Timestamp('2020-01-01 23:00:00+0000', tz='UTC')

In [9]:
# index_list = energy_consumption_dfs['hourly'][building_name].index

# enum_list = np.zeros(num_hours, dtype=int)

# for a in anomalies[building_name]:
#     mask = np.logical_and(a.start <= index_list, index_list <= a.end)
#     enum_list = enum_list + mask

# print(enum_list)
# print(np.count_nonzero(enum_list))
# print(enum_list.shape[0] - np.count_nonzero(enum_list))

In [10]:
index_list = energy_consumption_dfs['daily'][building_name].index
print(type(index_list))
print(index_list)

enum_list = np.zeros(len(index_list), dtype=int)

for a in anomalies[building_name]:
    i = index_list.get_loc(a.start)
    enum_list[i] = 1

print(enum_list)
print(np.count_nonzero(enum_list))
print(enum_list.shape[0] - np.count_nonzero(enum_list))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2019-01-01 00:00:00+00:00', '2019-01-02 00:00:00+00:00',
               '2019-01-03 00:00:00+00:00', '2019-01-04 00:00:00+00:00',
               '2019-01-05 00:00:00+00:00', '2019-01-06 00:00:00+00:00',
               '2019-01-07 00:00:00+00:00', '2019-01-08 00:00:00+00:00',
               '2019-01-09 00:00:00+00:00', '2019-01-10 00:00:00+00:00',
               ...
               '2019-12-23 00:00:00+00:00', '2019-12-24 00:00:00+00:00',
               '2019-12-25 00:00:00+00:00', '2019-12-26 00:00:00+00:00',
               '2019-12-27 00:00:00+00:00', '2019-12-28 00:00:00+00:00',
               '2019-12-29 00:00:00+00:00', '2019-12-30 00:00:00+00:00',
               '2019-12-31 00:00:00+00:00', '2020-01-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=366, freq='D')
[1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0

In [11]:
# index_list = energy_consumption_dfs['hourly'][building_name].index

# enum_list = np.zeros(num_hours, dtype=int)

# for a in anomalies[building_name]:
#     mask = np.logical_and(a.start <= index_list, index_list <= a.end)
#     enum_list = enum_list + mask

# print(enum_list)
# print(np.count_nonzero(enum_list))
# print(enum_list.shape[0] - np.count_nonzero(enum_list))

In [12]:
index_list = energy_consumption_dfs['hourly'][building_name].index

enum_list = np.zeros(len(index_list), dtype=int)

for a in anomalies[building_name]:
    mask = np.logical_and(a.start <= index_list, index_list < a.end)
    enum_list = enum_list + mask

signal_list = energy_consumption_dfs['hourly'][building_name]['Totalt'].to_list() * enum_list
signal_list = [x if x != 0 else None for x in signal_list]

enum_list = [int(x) for x in enum_list]

t = index_list.values.tolist() #[x.to_pydatetime() for x in index_list]

In [13]:
building_name_underscore = building_name.lower().replace(' ', '_')
enum_id = f'{building_name_underscore}_anomaly_enum'
enum_name = f'{building_name} anomaly enum'
enum_metadata = SignalInfo(name=enum_name, description='', type='enum', enumValues={0: ' ', 1: 'anomaly'}, labels={"building": [building_name]})

overlay_id = f'{building_name_underscore}_anomaly_overlay'
overlay_name = f'{building_name} anomaly overlay'
overlay_metadata = SignalInfo(name=overlay_name, description='', labels={"building": [building_name]}, gapDetection="PT1H", engUnit="kWh")

resp = clarify_client.save_signals(input_ids=[enum_id, overlay_id], signals=[enum_metadata, overlay_metadata])
t = index_list.values.tolist() #[x.to_pydatetime() for x in index_list]
data = DataFrame(times=t, series={enum_id: enum_list, overlay_id: signal_list})

clarify_client.insert(data)

Response(jsonrpc='2.0', id='1', result=InsertResponse(signalsByInput={'blomsterbyen_barnehage_anomaly_enum': InsertSummary(id='c9kkia4plln469u58mcg', created=False), 'blomsterbyen_barnehage_anomaly_overlay': InsertSummary(id='c9knd9kplln469u58mk0', created=False)}), error=None)

In [14]:
print(index_list)
print(t[0])

DatetimeIndex(['2019-01-01 00:00:00+00:00', '2019-01-01 01:00:00+00:00',
               '2019-01-01 02:00:00+00:00', '2019-01-01 03:00:00+00:00',
               '2019-01-01 04:00:00+00:00', '2019-01-01 05:00:00+00:00',
               '2019-01-01 06:00:00+00:00', '2019-01-01 07:00:00+00:00',
               '2019-01-01 08:00:00+00:00', '2019-01-01 09:00:00+00:00',
               ...
               '2020-01-01 14:00:00+00:00', '2020-01-01 15:00:00+00:00',
               '2020-01-01 16:00:00+00:00', '2020-01-01 17:00:00+00:00',
               '2020-01-01 18:00:00+00:00', '2020-01-01 19:00:00+00:00',
               '2020-01-01 20:00:00+00:00', '2020-01-01 21:00:00+00:00',
               '2020-01-01 22:00:00+00:00', '2020-01-01 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=8783, freq=None)
1546300800000000000
